In [15]:
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import datetime as dt
import time

In [16]:
# portfolio of crypto
crypto = ['BNB-USD', 'ETH-USD', 'BTC-USD', 'DOGE-USD', 'LINK-USD', 'SOL-USD', 'ADA-USD', 'XRP-USD', 'LTC-USD', 'CRO-USD']

In [17]:
# import yfinance as yf

# start_date = "2020-01-01"
# end_date = datetime.now().strftime("%Y-%m-%d")

# # Download historical data from yf API
# data = yf.download(crypto, start=start_date, end=end_date, group_by='ticker')

# # download data to csv
# data.to_csv("crypto.csv")

In [18]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv("crypto.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Forward fill missing values for each column
df = df.fillna(method='ffill')

# Convert all values to float
df = df.astype(float)

# rename tickers to more readable names
df.columns = pd.MultiIndex.from_tuples([
    (
        col[0].replace('BNB-USD', 'BNB').replace('ETH-USD', 'ETH').replace('BTC-USD', 'BTC').replace('DOGE-USD', 'DOGE').replace('LINK-USD', 'LINK').replace('SOL-USD', 'SOL').replace('ADA-USD', 'ADA').replace('XRP-USD', 'XRP').replace('LTC-USD', 'LTC').replace('CRO-USD', 'CRO'),
        col[1]
    ) for col in df.columns
])


# Show the result
# df.head()

In [19]:
# keep only 'Close' from MultiIndex Level 1
close_prices = df.xs('Close', level=1, axis=1)

# date index to datetime
close_prices.index = pd.to_datetime(close_prices.index)

# Show the result
close_prices.head()


,XRP,ETH,LTC,BNB,DOGE,BTC,ADA,LINK,SOL,CRO
Date,,,,,,,,,,
2020-01-01,0.192667,130.802002,42.017719,13.689083,0.002033,7200.174316,0.033458,1.806640,NaN,0.033973
2020-01-02,0.188043,127.410179,39.823013,13.027011,0.002009,6985.470215,0.032751,1.741144,NaN,0.032858
2020-01-03,0.193521,134.171707,42.415573,13.660452,0.002145,7344.884277,0.034180,1.809708,NaN,0.034666
2020-01-04,0.194355,135.069366,43.326607,13.891512,0.002241,7410.656738,0.034595,1.840133,NaN,0.034689
2020-01-05,0.195537,136.276779,43.553207,14.111019,0.002419,7411.317383,0.034721,1.807787,NaN,0.034618


In [20]:
# crypto ratios list
ratios_list = ['ETH/BTC', 'BNB/BTC', 'DOGE/BTC', 'SOL/BTC', 'ADA/BTC', 'XRP/BTC', 'LTC/BTC', 'CRO/BTC', 'LINK/BTC', 'SOL/ETH', 'LINK/ETH']

In [21]:
# create ratios DataFrame
ratios_df = pd.DataFrame(index=close_prices.index)

# Calculate the price ratios for each pair in the ratios list
for pair in ratios_list:
    base, quote = pair.split('/')
    ratios_df[pair] = close_prices[base] / close_prices[quote]

# fill any NaN values with 0 (zero)
"""
Although this is not the proper solution, it is a quick fix to avoid issues with NaN values, 
especially for SOL, that is more recent than the other cryptos.
"""
ratios_df = ratios_df.fillna(0)

# Show the result
ratios_df.head()

,ETH/BTC,BNB/BTC,DOGE/BTC,SOL/BTC,ADA/BTC,XRP/BTC,LTC/BTC,CRO/BTC,LINK/BTC,SOL/ETH,LINK/ETH
Date,,,,,,,,,,,
2020-01-01,0.018167,0.001901,2.823543e-07,0.0,0.000005,0.000027,0.005836,0.000005,0.000251,0.0,0.013812
2020-01-02,0.018239,0.001865,2.875970e-07,0.0,0.000005,0.000027,0.005701,0.000005,0.000249,0.0,0.013666
2020-01-03,0.018267,0.001860,2.920400e-07,0.0,0.000005,0.000026,0.005775,0.000005,0.000246,0.0,0.013488
2020-01-04,0.018226,0.001875,3.024024e-07,0.0,0.000005,0.000026,0.005847,0.000005,0.000248,0.0,0.013624
2020-01-05,0.018388,0.001904,3.263927e-07,0.0,0.000005,0.000026,0.005877,0.000005,0.000244,0.0,0.013266
